In [88]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
toy_data = load_data('TOY', 'spacy')

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.42 seconds


In [86]:
vocab = toy_data['vocab']
instances = [toy_data['data'][1][i] for i in range(2)]

In [91]:
bt = BackTrans()

Using cache found in /disks/sdb/torch_home/hub/pytorch_fairseq_master
Using cache found in /disks/sdb/torch_home/hub/pytorch_fairseq_master


In [96]:
bt.z2en.translate("dd")

'"I don\'t think it\'s fair," he said, "but I think it\'s fair.'

In [97]:
from allennlp.data.batch import Batch
print(Batch(instances).as_tensor_dict())

new_instances = toy_data['reader'].transform_instances(bt, instances)
batch = Batch(new_instances)
batch.index_instances(vocab)
print(batch.as_tensor_dict(batch.get_padding_lengths()))

{'sent': {'tokens': {'tokens': tensor([[14165,  3203,     5,  1675,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  879,    96,     8,  1026,    12,  8067,    11, 13591,     8, 14166,
             4,   673,   662,    15,     4,  1154,   240,   639,   417,     7]])}}, 'label': tensor([0, 1])}
{'sent': {'tokens': {'tokens': tensor([[    1,     3,  8423,     5,  1675,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  282,  1026,    12,  8067,    11,  3142,     8, 14166,    27,     4,
           108,   662,    15,     4,  1154,    23,   639,   417,     7]])}}, 'label': tensor([0, 1])}


In [83]:
coll = partial(transform_collate, toy_data['reader'], vocab, Identity())
coll(instances)

{'sent': {'tokens': {'tokens': tensor([[14165,  3203,     5,  1675,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  879,    96,     8,  1026,    12,  8067,    11, 13591,     8, 14166,
             4,   673,   662,    15,     4,  1154,   240,   639,   417,     7]])}}, 'label': tensor([0, 1])}


NameError: name 'exit' is not defined

In [144]:
dev_data =  toy_data['data'][1]
vocab

Vocabulary with namespaces:  tokens, Size: 11 || token_characters, Size: 18 || labels, Size: 3 || Non Padded Namespaces: {'*labels', '*tags'}

In [52]:
val_data = toy_data['data'][1]
from allennlp.data.dataset_readers.dataset_reader import AllennlpDataset
ds = AllennlpDataset(val_data, toy_data['vocab'])

In [72]:
ds.instances[5].fields['sent'].index(ds.vocab)

TypeError: tokens_to_indices() takes 3 positional arguments but 4 were given

In [32]:
config = Config()
task = Task(config)
task.from_pretrained()
config

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.36 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0123 seconds


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=transfer
	--adv_data=nogit/SST-lstm.hotflip.adv.tsv
	--transform=embed_aug
	--randomness=False
	--aug_data=
	--attack_method=hotflip
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=400
	--attack_gen_aug=False
	--attack_gen_adv=True
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm
	--tokenizer=spacy

In [41]:
config.attack_size = 50
config.attack_method = "pwws"
task.attack()

cache for SST-counter.vec exists
*** load SST-counter.vec from cache cost 0.0135 seconds



  0%|          | 0/50 [00:00<?, ?it/s]

Sanity Test



  8%|▊         | 4/50 [00:01<00:16,  2.84it/s]

[raw] the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .
	 ['  0.0303', '  0.9697']
[adv] the acting , costumes , music , cinematography nor sound are all astounding given the production 's austere locales .
	 ['  0.8179', '  0.1821']
[changed] 1

Avg.change# 1.0 Avg.change% 5.0
Aggregated metric: Accu before: 100.00%, after: 66.67%, Flip ratio 33.33%


...

[raw] a synthesis of cliches and absurdities that seems positively decadent in its cinematic flash and emptiness .
	 ['  0.9636', '  0.0364']
[adv] a synthesis of sociological and absurdities that seems positively decadent in its cinematic wink and emptiness .
	 ['  0.4770', '  0.5230']
[changed] 2

Avg.change# 1.68 Avg.change% 8.35
Aggregated metric: Accu before: 86.96%, after: 45.65%, Flip ratio 47.50%



 94%|█████████▍| 47/50 [00:22<00:01,  2.24it/s]

[raw] a subtle and well - crafted ( for the most part ) chiller .
	 ['  0.0015', '  0.9985']
[adv] a subtle nor well - crafted ( for the most part ) chiller .
	 ['  0.7256', '  0.2744']
[changed] 1

Avg.change# 1.65 Avg.change% 8.29
Aggregated metric: Accu before: 87.23%, after: 44.68%, Flip ratio 48.78%



100%|██████████| 50/50 [00:24<00:00,  2.04it/s]

[raw] it feels like an after - school special gussied up with some fancy special effects , and watching its rote plot points connect is about as exciting as gazing at an egg timer for 93 minutes .
	 ['  0.9829', '  0.0171']
[adv] it discoveries like an after - school unique gussied up with some fancy special effects , and watching its rouge plot points connect is about as exciting as gazing at an egg timer for 93 record .
	 ['  0.2827', '  0.7173']
[changed] 4

Avg.change# 1.76 Avg.change% 8.41
Aggregated metric: Accu before: 88.00%, after: 46.00%, Flip ratio 47.73%
Avg.change# 1.76 Avg.change% 8.41
Accu before: 88.00%, after: 46.00%, Flip ratio 47.73%


In [18]:
scores = {}
stops = stopwords.words("english")
all_words = list(task.vocab.get_index_to_token_vocabulary().values())[2:]
for word in all_words:
    if "'" in word:
        continue
    scores[word] = round(task.predictor.predict(word)['probs'][0], 2)

In [130]:
# from awesome_glue.utils import FreqUtil
# FreqUtil.split_by_frequency(task.vocab, 19, 20)
task.vocab._retained_counter['tokens']['Microsoft']

6375

TypeError: 'Vocabulary' object is not subscriptable

In [75]:
total = 0
for i in range(30):
    sent = allenutil.as_sentence(task.dev_data[i])
    before = task.predictor.predict(sent)['probs'][0]
    if before > 0.5:
        sent = "above above " + sent
    else:
        sent = "too too " + sent
    after = task.predictor.predict(sent)['probs'][0]
    if (before - 0.5) * (after - 0.5) < 0:
        total += 1
print(total)

11


In [27]:
sorted(scores.items(), key=lambda x:x[1], reverse=False)[:5000][-1]

('voices', 0.13)

In [145]:
task.vocab.get_token_index('d')

12732